In [3]:
import sys
sys.path.insert(0, "../../")


In [1]:
import os 
import frontmatter
from dataclasses import dataclass, field

def read_frontmater(file_path):
    # Create temp file

    with open(test_file_path) as f:
        post = frontmatter.load(f)

    fm = {key: post.get(key) for key in post.keys()}
    fm.update({'urls': []})

    for kb in fm.get('linked_kbs'):
        # url = re.search("(?P<url>https?://[^\s]+)", kb).group("url")
        t = kb.split('](')
        t = t[0][1:]
        fm['urls'].append(t)

    del fm['linked_kbs']

    return fm


test_file_path = "kb_md/posts/10000ft_connector/index.qmd"

class PathExists(Exception):
    def __init__(self):
        super().__init__("path does not exist")


if not os.path.exists(test_file_path):
    raise  PathExists(Exception)
    
test_fm = read_frontmater(test_file_path)
test_fm.keys()

dict_keys(['title', 'url', 'article_id', 'views', 'created_date', 'last updated', 'urls'])

In [4]:
import datacrew.crawler.crawler as dcc
driver = dcc.driversetup(is_headless=False)


In [71]:
from selenium.webdriver.common.by import By

TOPIC_URL = "https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC/connecting-data-to-domo?language=en_US"
TOPIC_URL = 'https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US'

BASE_URL = "https://domo-support.domo.com/s/topic/"

test_page_source = dcc.pagesource(url=TOPIC_URL, driver=driver,
                                  element_type=By.CLASS_NAME, element_id="section-list-item"
                                  )

# test_page_source


💤 loading https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US 💤
Page https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US is loaded within 10 seconds.


In [72]:
import datacrew.crawler.article as dca
import selenium.webdriver
from bs4 import BeautifulSoup


@dataclass(init=False)
class Article_Category(dca.Article):
    url: str
    base_url: str
    driver: selenium.webdriver = field(repr = False)

    is_success: bool = False
    article: dca.Article = field(default=None, repr=False)

    category: str = None
    category_id: str = None
    category_description: str = None

    child_category_ls: list[dict] = None

    def __init__(self, url, base_url, driver):
        self.url = url
        self.base_url = base_url
        self.driver = driver

        soup = dcc.pagesource(driver=self.driver, url=self.url,
                              element_type=By.CLASS_NAME, element_id="section-list-item")

        if not soup:
            raise dcc.ArticleKB_GetSoupError(url=self.url)

        super().__init__(base_url=base_url, soup=soup)

        self.article = dca.Article(soup=soup, base_url=self.base_url)
        
        self.set_category_id()

        self.kb_url_ls = self.article.linked_url_ls

        try:
            self.process_kb_soup(soup)
            self.is_success = True

        except dca.ArticleKB_ProcessSoupError as e:
            print(e)

    def set_category_id(self):
        url_str = self.url.replace( self.base_url , '')
        url_str = url_str.split('/')[-2]
        self.category_id = url_str
        return url_str

    def process_kb_soup(self, soup: BeautifulSoup):

        # process parent attributes
        parent_term = "page-header"
        parent = soup.find(class_ = [parent_term])
        self.category = parent.find("h1").get_text()

        self.category_description = parent.find(
            "p") and parent.find("p").get_text()


        # table_term = "section-list"
        table_item_term = "section-list-item"

        table = soup.find_all(class_=[table_item_term])

        if not table or table == []:
            raise dca.ArticleKB_ProcessSoupError(
                url=self.url, search_term=table_term)

        tarticle = []
        for row in table:
            url = row.find("a").get("href")
            if not url.startswith('/s/topic/'):
                continue

            id = url.split('/')[-1]

            tarticle.append({'category' : row.get_text(),
            'id' :  id,
            'url' : f"{self.base_url}{id}"}
            )
        
        self.child_category_ls = tarticle

In [73]:
from pprint import pprint

TOPIC_URL = 'https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US'

pprint(Article_Category(url=TOPIC_URL,
                 driver=driver,
                 base_url=BASE_TOPIC_URL))

💤 loading https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US 💤
Page https://domo-support.domo.com/s/topic/0TO5w000000ZanLGAS/connectors?language=en_US is loaded within 10 seconds.
Article_Category(base_url='https://domo-support.domo.com/s/topic/',
                 linked_url_ls=['https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZaoQGAS',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZapAGAS',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZaoVGAS',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZaojGAC',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZaoyGAC',
                                'https://domo-support.domo.com/s/topic/0TO5w000000ZapVGAS',
                                'https://domo-support.domo.com/s/topic/0TO5w

In [ ]:
import os

kb_path = "nbs/projects/kb_md/posts"
kb_path = "kb_md/posts/"

if not os.path.exists(kb_path):
    raise Exception('kb_path does not exist')

for path, subdirs, files in os.walk(kb_path):
    for name in files:
        if name != 'index.qmd':
            continue
        # print(path, name)
        file_path = os.path.join(path, name)


In [42]:
test_str = 'https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC/connecting-data-to-domo?language=en_US'

BASE_TOPIC_URL


    
clean_url(test_str)

'0TO5w000000ZammGAC'